# Migrating R libraries R

*Note:* it is safe to run this notebook several times. If no actions are required, then nothing will be done.

In [ ]:
function list_old_r_versions() {
    find ${R_LIBS_DIR} -mindepth 1 -maxdepth 1 -type d -exec basename {} \; | grep -v "${NEW}" | sort
}

function list_libs() {
    # First parameter: R versions (e.g. 3.8)
    # Second optional parameter: 0 or 1 (create or not a R lib directory)
    local version=$1
    local do_create=${2:-0}
    [[ ${do_create} -eq 1 ]] && mkdir -p ${R_LIBS_DIR}/${version}/lib
    local path="${R_LIBS_DIR}/${version}/lib"
    if [[ -d ${path} ]]; then
        find ${path} -maxdepth 1 -mindepth 1 -type d | rev | cut -d '/' -f 1 | rev
    else
        return 0
    fi
}

function set_variables() {
    MIRROR="https://stat.ethz.ch/CRAN/"
    R_LIBS_DIR="/home/.local/R"
    NEW="$(R --version | head -n 1 | cut -d ' ' -f 3 | cut -d '.' -f 1-2)"
    
    if [[ $(list_old_r_versions | wc -l) -eq 0 ]]; then
        echo "No old R versions found !"
        return 0
    fi
    if [[ -z ${NEW} ]]; then
        echo "Current R version could not be determined!"
        return 1
    fi
    echo "Old R versions found:"
    list_old_r_versions
    echo "Current R version is:"
    echo ${NEW}
    echo "Package Mirror:"
    echo ${MIRROR}
    return 0
}

set_variables

## Did you install extra R libraries?

Let's check that! Run the next command to list the installed R libraries that need migrating:

In [ ]:
function check_libs_to_migrate() {
    set_variables > /dev/null
    [[ $? -eq 1 ]] && return 1
    for version in $(list_old_r_versions); do
        echo "Checking what libs need to be migrated from R version ${version} to R version ${NEW}:"
        list_libs ${version}
    done
    echo "All done!"
}

check_libs_to_migrate

If the output of the previous cell is empty, it means that you do not have installed any R libraries. **You are done with this guide!**

If the cell output contains a list of libraries, then read on to re-install those libraries!

## Let's re-install the R libraries

By running the following cell, all the existing old R libraries will be installed again in your new R environment.

In [ ]:
function run_r_command() {
        Rscript --slave --no-save --no-restore-history -e "$@"
}

function migrate_libs() {
    set_variables > /dev/null
    [[ $? -eq 1 ]] && return 1
    for version in $(list_old_r_versions); do
        echo "Dealing with R version ${version}..."
        for lib in $(list_libs ${version}); do
            echo "* Processing ${lib} (from R ${version})"
            if [[ $(list_libs ${NEW} 1 | grep -c "^${lib}$") -eq 1 ]]; then
                echo "Library ${lib} has already been migrated to R ${NEW}"
            else
                cmd="install.packages(pkgs='${lib}', repos=c('${MIRROR}'), lib='~/.local/R/${NEW}/lib');"
                time run_r_command "$cmd"
            fi
            echo
        done
        echo "Done for R version ${version}"
    done
    echo "All done!"
}

migrate_libs

## Clean-up

Once all R libraries have been migrated, you are ready to remove the old R libraries.
Run the following cell to remove all libraries that have been migrated.

In [ ]:
function cleanup_libs() {
    set_variables > /dev/null
    [[ $? -eq 1 ]] && return 1
    for version in $(list_old_r_versions); do
        for lib in $(list_libs ${version}); do
            echo "* Processing ${lib} (R ${version})"
            if [[ $(list_libs ${NEW} 1 | grep -c "^${lib}$") -eq 1 ]]; then
                echo "Library ${lib} has already been migrated to R ${NEW} => Removing it"
                rm -rf /home/.local/R/${version}/lib/${lib}
            else
                echo "Library ${lib} has NOT been migrated to R ${NEW}. Remove it manually if you don't need it:"
                echo "rm -rf /home/.local/R/${version}/lib/${lib}"
            fi
        done
        if [[ -d /home/.local/R/${version}/lib && -n "$(find /home/.local/R/${version}/lib -maxdepth 0 -type d -empty 2> /dev/null)" ]]; then
        echo "No more R ${version} libraries found. Removing /home/.local/R/${version}."
        rm -rf /home/.local/R/${version}
        fi
    done
    echo "All done!"
}

cleanup_libs

**End of notebook**